<a href="https://colab.research.google.com/github/niran1915/gen-ai-assignment-2/blob/main/PES2UG23CS308_GENAI_assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import pipeline
import pandas as pd

# 1. Load the Toxicity Pipeline
# We use 'unitary/toxic-bert' which is fine-tuned on the Jigsaw Toxic Comment dataset.
# device=0 uses the GPU if you have enabled it in Colab (Runtime > Change runtime type > GPU)
print("Loading model... this may take a minute.")
classifier = pipeline("text-classification", model="unitary/toxic-bert", top_k=None)

def analyze_comment(comment, threshold=0.7):
    """
    Analyzes a comment and flags it if any toxicity score exceeds the threshold.
    """
    results = classifier(comment)[0]

    # The model returns scores for: toxic, severe_toxic, obscene, threat, insult, identity_hate
    # We filter for any label with a score higher than our confidence threshold
    flagged_labels = [
        f"{res['label']} ({res['score']:.2f})"
        for res in results
        if res['score'] > threshold
    ]

    is_toxic = len(flagged_labels) > 0

    return {
        "text": comment,
        "is_toxic": "🚩 FLAG" if is_toxic else "✅ SAFE",
        "details": flagged_labels if is_toxic else ["Clean"]
    }

# 3. Test Data (Simulation of Forum Comments)
forum_comments = [
    "This is a really helpful tutorial, thanks!",
    "You are extremely stupid and you should delete your account.",
    "I disagree with your point about python, but good effort.",
    "I'm going to hurt you if you post this again."
]

# 4. Run the Detector
print(f"\n{' STATUS ': <10} | {' COMMENT ': <50} | DETAILS")
print("-" * 80)

for comment in forum_comments:
    result = analyze_comment(comment)
    print(f"{result['is_toxic']: <10} | {result['text'][:47]+'...': <50} | {result['details']}")

Loading model... this may take a minute.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu



 STATUS    |  COMMENT                                           | DETAILS
--------------------------------------------------------------------------------
✅ SAFE     | This is a really helpful tutorial, thanks!...      | ['Clean']
🚩 FLAG     | You are extremely stupid and you should delete ... | ['toxic (0.98)', 'insult (0.91)']
✅ SAFE     | I disagree with your point about python, but go... | ['Clean']
✅ SAFE     | I'm going to hurt you if you post this again....   | ['Clean']


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Ensure the classifier is loaded from the previous step
if 'classifier' not in locals():
    print("Please run the model loading cell first!")
else:
    def gradio_interface(text):
        results = classifier(text)[0]
        # Format the output for the interface
        # We create a dictionary of {Label: Score}
        output_scores = {res['label']: res['score'] for res in results}
        return output_scores

    # Create the app
    demo = gr.Interface(
        fn=gradio_interface,
        inputs=gr.Textbox(lines=2, placeholder="Type a comment here to test toxicity..."),
        outputs=gr.Label(num_top_classes=3),
        title="Cyberbullying Detector 🛡️",
        description="Enter a comment to see if the AI flags it as toxic, threat, or insult."
    )

    # Launch it directly in the notebook
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c988fb9b0dc4965630.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
